In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import numpy as np
# 設定具體的開始和結束日期
start_date = '2005-04-01'
end_date = '2024-09-30'

# 下載台灣加權指數資料
twii = yf.download('^TWII', start=start_date, end=end_date)

# 先檢查列名結構
print("列名結構：")
print(twii.columns)

# 從索引中提取年份和月份
twii['年'] = twii.index.year
twii['月'] = twii.index.month

# 計算日報酬率
twii['Return'] = twii['Adj Close'].pct_change()
twii['Return'] = twii['Return'].fillna(0)
# 計算月度資料 - 使用正確的多層索引列名
monthly_twii = twii.groupby(['年', '月']).agg({
    ('Adj Close', '^TWII'): 'last',
    ('Return', ''): lambda x: (1 + x).prod() - 1
}).reset_index()

# 轉換為百分比格式
monthly_twii[('Return', '')] = monthly_twii[('Return', '')] * 100

# 重命名欄位
monthly_twii = monthly_twii.rename(columns={
    ('Adj Close', '^TWII'): '月底收盤價',
    ('Return', ''): '月報酬率(%)'
})
# 創建年月欄位，格式為 YYYY/MM
monthly_twii['年月'] = monthly_twii['年'].astype(str) + '/' + monthly_twii['月'].astype(str).str.zfill(2)
# 修正多層索引列名為單層索引
monthly_twii.columns = [col[0] if isinstance(col, tuple) else col for col in monthly_twii.columns]


# 顯示結果
print(monthly_twii.head())

[*********************100%***********************]  1 of 1 completed

列名結構：
MultiIndex([('Adj Close', '^TWII'),
            (    'Close', '^TWII'),
            (     'High', '^TWII'),
            (      'Low', '^TWII'),
            (     'Open', '^TWII'),
            (   'Volume', '^TWII')],
           names=['Price', 'Ticker'])
      年  月    Adj Close    Return       年月
0  2005  4  5818.043457 -3.494583  2005/04
1  2005  5  6011.532227  3.325667  2005/05
2  2005  6  6241.911133  3.832283  2005/06
3  2005  7  6311.950684  1.122085  2005/07
4  2005  8  6033.442383 -4.412397  2005/08


In [2]:
# 下載美國一個月期國庫券利率數據（通常用作無風險利率）
rf = yf.download('^IRX', start=start_date, end=end_date)

# 從索引中提取年份和月份
rf['年'] = rf.index.year
rf['月'] = rf.index.month

# 計算月度資料
monthly_rf = rf.groupby(['年', '月']).agg({
    ('Adj Close', '^IRX'): 'last'  # 使用月底利率
}).reset_index()

# 將年利率轉換為月利率 (除以12) 並轉換為小數
monthly_rf[('Adj Close', '^IRX')] = monthly_rf[('Adj Close', '^IRX')] / (12 * 100)

# 重命名欄位
monthly_rf = monthly_rf.rename(columns={
    ('Adj Close', '^IRX'): '無風險月報酬率'
})
# 創建年月欄位，格式為 YYYY/MM
monthly_rf['年月'] = monthly_rf['年'].astype(str) + '/' + monthly_rf['月'].astype(str).str.zfill(2)
# 修正多層索引列名為單層索引
monthly_rf.columns = [col[0] if isinstance(col, tuple) else col for col in monthly_rf.columns]

# 顯示結果
print(monthly_rf.head())



[*********************100%***********************]  1 of 1 completed

      年  月  Adj Close       年月
0  2005  4   0.002364  2005/04
1  2005  5   0.002402  2005/05
2  2005  6   0.002550  2005/06
3  2005  7   0.002777  2005/07
4  2005  8   0.002858  2005/08
